## Core

> The building blocks to the UI

In [ ]:
#| default_exp core

## Imports

In [ ]:
#| hide
#| export
import fasthtml.common as fh
from monsterui.foundations import *
from fasthtml.common import is_listy, Div, P, Span, Script, FastHTML, FT, to_xml, show,fast_app
from fasthtml.svg import Svg
from fasthtml.components import Uk_theme_switcher, Main
from enum import Enum, auto
from fasthtml.components import Uk_select,Uk_input_tag,Uk_icon
from functools import partial
from itertools import zip_longest
from typing import Union, Tuple, Optional
from fastcore.all import *
import copy, re, httpx
from pathlib import Path

In [ ]:
from IPython.display import HTML, display, Markdown
from fasthtml.jupyter import *
from functools import partial

In [ ]:
from nbdev.showdoc import *

## App

In [ ]:
#| export
@delegates(fh.fast_app, but=['pico'])
def fast_app(*args, pico=False, **kwargs):
    "Create a FastHTML or FastHTMLWithLiveReload app with `bg-background text-foreground` to bodykw for frankenui themes"
    if 'bodykw' not in kwargs: kwargs['bodykw'] = {}
    if 'class' not in kwargs['bodykw']: kwargs['bodykw']['class'] = ''
    kwargs['bodykw']['class'] = stringify((kwargs['bodykw']['class'],'bg-background text-foreground'))
    return fh.fast_app(*args, pico=pico, **kwargs)

In [ ]:
assert fast_app(bodykw={'something':'test'})[0].bodykw == {'something': 'test', 'class': ' bg-background text-foreground'}

In [ ]:
#| export
@delegates(fh.FastHTML, but=['pico'])
def FastHTML(*args, pico=False, **kwargs):
    "Create a FastHTML app and adds `bg-background text-foreground` to bodykw for frankenui themes"
    if 'bodykw' not in kwargs: kwargs['bodykw'] = {}
    if 'class' not in kwargs['bodykw']: kwargs['bodykw']['class'] = ''
    kwargs['bodykw']['class'] = stringify((kwargs['bodykw']['class'],'bg-background text-foreground'))
    bodykw = kwargs.pop('bodykw',{})
    return fh.FastHTML(*args, pico=pico, **bodykw, **kwargs)

In [ ]:
FastHTML(bodykw={'something':'test'}).bodykw

{'pico': False, 'something': 'test', 'class': ' bg-background text-foreground'}

## Theme / Headers

You can select a theme color to and get all the headers.

In [ ]:
#| export
def _headers_theme(color, mode='auto'):
    mode_script = {
        'auto': '''
            if (
                localStorage.getItem("mode") === "dark" ||
                (!("mode" in localStorage) &&
                window.matchMedia("(prefers-color-scheme: dark)").matches)
            ) {
                htmlElement.classList.add("dark");
            } else {
                htmlElement.classList.remove("dark");
            }
        ''',
        'light': 'htmlElement.classList.remove("dark");',
        'dark': 'htmlElement.classList.add("dark");'
    }
    
    return fh.Script(f'''
        const htmlElement = document.documentElement;
        {mode_script[mode]}
        htmlElement.classList.add(localStorage.getItem("theme") || "uk-theme-{color}");
    ''')

In [ ]:
#| export
HEADER_URLS = {
        'franken_css': "https://unpkg.com/franken-ui@1.1.0/dist/css/core.min.css",
        'franken_js': "https://unpkg.com/franken-ui@1.1.0/dist/js/core.iife.js",
        'icon_js': "https://cdn.jsdelivr.net/gh/answerdotai/monsterui@main/monsterui/icon.iife.js",
        'tailwind': "https://cdn.tailwindcss.com",
        'daisyui': "https://cdn.jsdelivr.net/npm/daisyui@4.12.22/dist/full.min.css"}

def _download_resource(url, static_dir):
    "Download a single resource and return its local path"
    static = Path(static_dir)
    fname = static/f"{url[0]}.{'js' if 'js' in url[1] else 'css'}"
    content = httpx.get(url[1], follow_redirects=True).content
    fname.write_bytes(content)
    return (url[0], f"/{static_dir}/{fname.name}")

In [ ]:
#| export
daisy_styles = Style("""
:root {
  --p: from hsl(var(--primary)) l c h;
  --pc: from hsl(var(--primary-foreground)) l c h;
  --s: from hsl(var(--secondary)) l c h;
  --sc: from hsl(var(--secondary-foreground)) l c h;
  --b2: from hsl(var(--card-background)) l c h;
  --b1: from hsl(var(--background)) l c h;
  --bc: from hsl(var(--foreground)) l c h;
  --b3: from hsl(var(--ring)) l c h;
  --er: from hsl(var(--destructive)) l c h;
  --erc: from hsl(var(--destructive-foreground)) l c h;
}
""")


In [ ]:
#| export
class Theme(Enum):
    "Selector to choose theme and get all headers needed for app.  Includes frankenui + tailwind"
    def _generate_next_value_(name, start, count, last_values): return name
    slate = auto()
    stone = auto()
    gray = auto()
    neutral = auto()
    red = auto()
    rose = auto()
    orange = auto()
    green = auto()
    blue = auto()
    yellow = auto()
    violet = auto()
    zinc = auto()


    def _create_headers(self, urls, mode='auto', daisy=True):
        "Create header elements with given URLs"
        hdrs = [
            fh.Link(rel="stylesheet", href=urls['franken_css']),
            fh.Script(type="module", src=urls['franken_js']),
            fh.Script(type="module", src=urls['icon_js']),
            fh.Script(src=urls['tailwind']),
            _headers_theme(self.value, mode=mode)]
        if daisy: 
            hdrs += [fh.Link(rel="stylesheet", href=urls['daisyui']),daisy_styles]
        return hdrs

    def headers(self, mode='auto', daisy=True):
        "Create frankenui and tailwind cdns"
        return self._create_headers(HEADER_URLS, mode=mode, daisy=daisy)
    
    def local_headers(self, mode='auto', static_dir='static'):
        "Create headers using local files downloaded from CDNs"
        Path(static_dir).mkdir(exist_ok=True)
        local_urls = dict([_download_resource(url, static_dir) for url in HEADER_URLS.items()])
        return self._create_headers(local_urls, mode=mode,daisy=daisy)

In [ ]:
hdrs = Theme.blue.headers()
app = FastHTML(hdrs=hdrs)

In [ ]:
app, rt = fast_app(hdrs=Theme.blue.headers())
Show = partial(HTMX, app=app)

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()